# Full text search and keyword association analysis

全文檢索與關聯分析

# Load preprocessed news dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('cna_news_preprocessed.csv',sep='|')

In [3]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,aipl_20220314_1,2022-03-14,政治,外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾,民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中...,0.01,"['外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外', '親赴外交部捐贈物資的民眾約173...","[('外交部', 14), ('民眾', 7), ('物資', 7), ('烏克蘭', 5)...","['民眾', '捐贈', '烏克蘭', '的', '愛心', '物資', '持續', '湧入...","['民眾', '烏克蘭', '愛心', '物資', '外交部', '收到', '外交部長',...","[NerToken(word='烏克蘭', ner='GPE', idx=(4, 7)), ...","[('民眾', 'Na'), ('捐贈', 'VD'), ('烏克蘭', 'Nc'), ('...",https://www.cna.com.tw/news/aipl/202203140364....,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...


# Filter data by searching keywords from "content" column

## "in" is very powerful

In [4]:
df.content[0]

'民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝。外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日。外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問。根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜。外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物。送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾。募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部。外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力。外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意。'

In [5]:
qk = '外交部長吳釗燮'
text = df.content[0]
qk in text

True

In [6]:
qk = '居家自主健康管理'
text = df.content[0]
qk in text

False

In [7]:
qk = '捐贈烏克蘭'
text = df.content[0]
qk in text

True

In [8]:
qk = '外交部'
text = df.content[0]
qk in text

True

### all() any()

In [9]:
user_keywords = ['捐贈烏克蘭','外交部']
text = df.content[0]
all((qk in text) for qk in user_keywords)

True

In [10]:
user_keywords = ['肺炎疫情全球延燒','居家自主健康管理']
text = df.content[0]
any((qk in text) for qk in user_keywords)

False

In [11]:
user_keywords = ['烏克蘭戰爭','外交部']
text = df.content[0]
any((qk in text) for qk in user_keywords)

True

### Using apply() and lambda function

In [12]:
# Use apply() and lambda function
user_keywords = ['捐贈烏克蘭','外交部']
df.content.apply(lambda text: all((qk in text) for qk in user_keywords))

0       True
1      False
2      False
3      False
4      False
       ...  
208    False
209    False
210    False
211    False
212    False
Name: content, Length: 213, dtype: bool

# Filter data using the following function

In [13]:
from datetime import datetime, timedelta

In [14]:
from datetime import datetime, timedelta
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_dataFrame_fullText(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: keywords 
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in user_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in user_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


In [15]:
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_dataFrame_fullText_v0(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # proceed filtering
    if (cate == "全部") & (cond == 'and'):
        df_query = df[(df.date >= start_date) & (df.date <= end_date) 
            & df.content.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cate == "全部") & (cond == 'or'):
        df_query = df[(df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.content.apply(lambda text: any((qk in text) for qk in user_keywords))]
    elif (cond == 'and'):
        df_query = df[(df.category == cate) 
            & (df.date >= start_date) & (df.date <= end_date) 
            & df.content.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cond == 'or'):
        df_query = df[(df.category == cate) 
            & (df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.content.apply(lambda text: any((qk in text) for qk in user_keywords))]

    return df_query

In [16]:
user_keywords = ['捐贈烏克蘭','外交部']
cond='and'
cate='全部'
weeks=2

df_query = filter_dataFrame_fullText(user_keywords, cond, cate,weeks)
df_query.shape

(1, 14)

In [17]:
user_keywords = ['烏克蘭']
cond='and'
cate='全部'
weeks=2

df_query = filter_dataFrame_fullText(user_keywords, cond, cate,weeks)
df_query.shape

(23, 14)

# Get news title, category, and link

In [18]:
user_keywords = ['疫情','疫苗']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)
len(df_query)

6

In [19]:
df_query

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
137,ahel_20220314_18,2022-03-14,生活,陳時中：國門解封戒慎不恐懼 檢疫10天應該安全,邊境管制3月7日起鬆綁，指揮中心指揮官陳時中今天說，入境檢疫10天應該安全，現在談全面解封言...,0.0,"['台灣3月7日起將入境檢疫期縮短至10天', '中央流行疫情指揮中心指揮官陳時中今天在記者...","[('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3),...","['邊境', '管制', '3月', '7日', '起', '鬆綁', '，', '指揮',...","['邊境', '鬆綁', '指揮', '中心', '指揮官', '陳時中', '入境', '...","[NerToken(word='3月7日', ner='DATE', idx=(4, 8))...","[('邊境', 'Nc'), ('管制', 'Nv'), ('3月', 'Nd'), ('7...",https://www.cna.com.tw/news/ahel/202203140235....,NaN
197,acn_20220314_5,2022-03-14,兩岸,香港確診增2.7萬例 本波死亡人數逾4千,香港今天新增COVID-19陽性病例2萬6908例，其中核酸確診1萬2040例，其餘1萬48...,1.0,"['其中默沙東藥廠的口服藥已開始處方給指定診所及安老院患者等', '首批輝瑞口服藥也會於今日...","[('香港', 6), ('疫情', 6), ('死亡率', 6), ('疫苗', 5), ...","['香港', '今天', '新增', 'COVID-19', '陽性', '病例', '2萬...","['香港', '陽性', '病例', '核酸', '確診', '快篩', '陽性', '疫情...","[NerToken(word='香港', ner='GPE', idx=(0, 2)), N...","[('香港', 'Nc'), ('今天', 'Nd'), ('新增', 'VJ'), ('C...",https://www.cna.com.tw/news/acn/202203140291.aspx,NaN
202,acn_20220314_10,2022-03-14,兩岸,香港疫情死亡率超英美韓星 專家：可預見的悲劇,香港第5波疫情死亡率超越英美韓星。香港大學專家分析，香港居住環境擠迫、無法將安老院與社區分隔...,1.0,"['香港居住環境擠迫、無法將安老院與社區分隔、長者疫苗接種率低及公營醫療資源不足', '但可...","[('香港', 16), ('疫情', 10), ('長者', 9), ('死亡率', 7)...","['香港', '第5', '波', '疫情', '死亡率', '超越', '英', '美',...","['香港', '疫情', '死亡率', '超越', '韓星', '香港', '大學', '專...","[NerToken(word='第5', ner='ORDINAL', idx=(2, 4)...","[('香港', 'Nc'), ('第5', 'Neu'), ('波', 'Nf'), ('疫...",https://www.cna.com.tw/news/acn/202203140181.aspx,NaN
206,acn_20220314_14,2022-03-14,兩岸,專家：香港疫情再橫行2至3週 清零有難度,香港最近的COVID-19單日確診仍處於約3萬例的高點，香港中文大學專家今天說，推算疫情會於...,0.0,"['5月底至6月初每天確診病例才有望回落至約1000例', '5月底至6月每天確診病例才有望...","[('香港', 5), ('疫情', 4), ('確診', 4), ('病例', 4), (...","['香港', '最近', '的', 'COVID-19', '單日', '確診', '仍',...","['香港', '高點', '香港', '中文', '大學', '專家', '推算', '疫情...","[NerToken(word='香港', ner='GPE', idx=(0, 2)), N...","[('香港', 'Nc'), ('最近', 'Nd'), ('的', 'DE'), ('CO...",https://www.cna.com.tw/news/acn/202203140112.aspx,NaN
209,acn_20220314_17,2022-03-14,兩岸,阻疫情 香港擬上門為長者殘障人士施打疫苗,香港的COVID-19（2019冠狀病毒疾病）疫情嚴峻，官員表示，考慮擴大家居接種服務，上門...,0.0,"['估計約有5萬名獨居長者或殘障人士需要家居接種服務', '本月下旬將上門為不曾接種疫苗的居...","[('香港', 7), ('人士', 6), ('施打', 6), ('疫苗', 6), (...","['香港', '的', 'COVID-19', '（', '2019', '冠狀病毒', '...","['香港', '冠狀病毒', '疾病', '疫情', '官員', '擴大', '接種', '...","[NerToken(word='香港', ner='GPE', idx=(0, 2)), N...","[('香港', 'Nc'), ('的', 'DE'), ('COVID-19', 'FW')...",https://www.cna.com.tw/news/acn/202203140064.aspx,NaN
211,acn_20220314_19,2022-03-14,兩岸,中國為何堅持疫情清零 專家指醫療資源不足不均,中國抗疫專家張文宏解釋為何中國當前對疫情仍須堅持「社會面清零」，因為各地無論是心理上還是社會...,0.0,"['為何中國人已經接種了這沒多疫苗還不能「躺平」（放任不管）', '「但是有事的絕大多數都是...","[('中國', 14), ('疫苗', 13), ('接種', 7), ('疫情', 6),...","['中國', '抗疫', '專家', '張文宏', '解釋', '為何', '中國', '當...","['中國', '專家', '張文宏', '中國', '疫情', '社會面', '心理', '...","[NerToken(word='中國', ner='GPE', idx=(0, 2)), N...","[('中國', 'Nc'), ('抗疫', 'VJ'), ('專家', 'Na'), ('張...",https://www.cna.com.tw/news/acn/202203140027.aspx,NaN


In [20]:
for i in range(3):   
    print(i)
    print(df_query.iloc[i]['category'])
    print(df_query.iloc[i]['title'])
    print(df_query.iloc[i].link)
    print(df_query.iloc[i].photo_link)

0
生活
陳時中：國門解封戒慎不恐懼  檢疫10天應該安全
https://www.cna.com.tw/news/ahel/202203140235.aspx
nan
1
兩岸
香港確診增2.7萬例 本波死亡人數逾4千
https://www.cna.com.tw/news/acn/202203140291.aspx
nan
2
兩岸
香港疫情死亡率超英美韓星 專家：可預見的悲劇
https://www.cna.com.tw/news/acn/202203140181.aspx
nan


In [21]:
df_query.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
137,ahel_20220314_18,2022-03-14,生活,陳時中：國門解封戒慎不恐懼 檢疫10天應該安全,邊境管制3月7日起鬆綁，指揮中心指揮官陳時中今天說，入境檢疫10天應該安全，現在談全面解封言...,0.0,"['台灣3月7日起將入境檢疫期縮短至10天', '中央流行疫情指揮中心指揮官陳時中今天在記者...","[('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3),...","['邊境', '管制', '3月', '7日', '起', '鬆綁', '，', '指揮',...","['邊境', '鬆綁', '指揮', '中心', '指揮官', '陳時中', '入境', '...","[NerToken(word='3月7日', ner='DATE', idx=(4, 8))...","[('邊境', 'Nc'), ('管制', 'Nv'), ('3月', 'Nd'), ('7...",https://www.cna.com.tw/news/ahel/202203140235....,NaN


## All-in-one function to return category, title, link, and photo_link

"photo_link" will be use in the next app

In [22]:
user_keywords = ['疫情','疫苗']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)

In [23]:
len(df_query)

6

In [24]:
# get titles and links from k pieces of news 
def get_title_link_topk(df_query, k=5):
    items = []
    for i in range( len(df_query[0:k]) ): # show only 5 articles
        category = df_query.iloc[i]['category']
        title = df_query.iloc[i]['title']
        link = df_query.iloc[i]['link']
        photo_link = df_query.iloc[i]['photo_link']
        # if photo_link value is NaN, replace it with empty string 
        if pd.isna(photo_link):
            photo_link='' # 若沒圖片，就設定為空字串，在前端網頁解讀json格式時才不會錯誤
        
        item_info = {
            'category': category, 
            'title': title, 
            'link': link, 
            'photo_link': photo_link
        }

        items.append(item_info)
    return items 

In [25]:
get_title_link_topk(df_query, 3)

[{'category': '生活',
  'title': '陳時中：國門解封戒慎不恐懼  檢疫10天應該安全',
  'link': 'https://www.cna.com.tw/news/ahel/202203140235.aspx',
  'photo_link': ''},
 {'category': '兩岸',
  'title': '香港確診增2.7萬例 本波死亡人數逾4千',
  'link': 'https://www.cna.com.tw/news/acn/202203140291.aspx',
  'photo_link': ''},
 {'category': '兩岸',
  'title': '香港疫情死亡率超英美韓星 專家：可預見的悲劇',
  'link': 'https://www.cna.com.tw/news/acn/202203140181.aspx',
  'photo_link': ''}]

### Some photo_links are "NaN". 

If the photo_link value is 'nan', it will be a problem when converted to json format on Django!

    "NaN" cannot be converted to a JSON string. 

In [26]:
df_query.iloc[0]['photo_link']

nan

### Test the photo_link is NaN or not and replace it with empty string
    How to do?
    You can test a variable is "NaN" or not by using pd, np or math.

In [27]:
import pandas as pd
import numpy as np
import math

# we can use pandas, numpy or math to check if photo_link is NaN
x = float("nan")

print(f"It's pd.isna  : {pd.isna(x)}")
print(f"It's np.isnan  : {np.isnan(x)}")
print(f"It's math.isnan : {math.isnan(x)}")

It's pd.isna  : True
It's np.isnan  : True
It's math.isnan : True


In [28]:
df_query.iloc[0]['photo_link']

nan

In [29]:
photo_link = df_query.iloc[0]['photo_link']
# if photo_link value is NaN, replace it with empty string 
if pd.isna(photo_link):
    photo_link='' # 

In [30]:
photo_link

''

# Find some related keywords

    Find related words from the top_key_freq column
    相關詞有哪一些? 找出各篇文章的topk關鍵詞?

## All-in-one function: Get related keywords

In [31]:
from collections import Counter
# 相關詞有哪一些?找出各篇文章的topk關鍵詞加以彙整計算
# 不能用 "get_related_keys"當函數名稱，因為這是Django系統用的名稱
def get_related_words(df_query):
    counter=Counter() # this counter is for all articles
    for idx in range(len(df_query)):
        pair_dict = dict(eval(df_query.iloc[idx].top_key_freq))
        counter += Counter(pair_dict)
    return counter.most_common(20) #return list format

In [32]:
# This version is for reference
def get_related_words_v0(df_query):
    all_pairs={}
    for idx in range(len(df_query)):
        row = df_query.iloc[idx].top_key_freq
        pairs = eval(row)
        for pair in pairs:
            w,f = pair
            if w in all_pairs:
                all_pairs[w]+= f
            else:
                all_pairs[w] = f

    counter = Counter(all_pairs)
    return counter.most_common(20) #return list format
    #return dict(counter.most_common(20)) #return dict format

In [33]:
user_keywords = ['疫情','疫苗']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)

In [34]:
df_query.shape

(6, 14)

In [35]:
get_related_words(df_query)

[('香港', 39),
 ('疫情', 34),
 ('疫苗', 32),
 ('接種', 20),
 ('中國', 16),
 ('長者', 15),
 ('死亡率', 13),
 ('病例', 12),
 ('確診', 8),
 ('病毒', 8),
 ('醫療', 8),
 ('陳時中', 7),
 ('安老院', 7),
 ('人士', 7),
 ('大學', 7),
 ('資源', 7),
 ('薛達', 6),
 ('施打', 6),
 ('解封', 5),
 ('開放', 5)]

In [36]:
# get_related_words_v1(df_query)

## A Step by step demonstration (do it yourself)

In [37]:
user_keywords = ['疫情','疫苗']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)

In [38]:
df_query.head(2)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
137,ahel_20220314_18,2022-03-14,生活,陳時中：國門解封戒慎不恐懼 檢疫10天應該安全,邊境管制3月7日起鬆綁，指揮中心指揮官陳時中今天說，入境檢疫10天應該安全，現在談全面解封言...,0.0,"['台灣3月7日起將入境檢疫期縮短至10天', '中央流行疫情指揮中心指揮官陳時中今天在記者...","[('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3),...","['邊境', '管制', '3月', '7日', '起', '鬆綁', '，', '指揮',...","['邊境', '鬆綁', '指揮', '中心', '指揮官', '陳時中', '入境', '...","[NerToken(word='3月7日', ner='DATE', idx=(4, 8))...","[('邊境', 'Nc'), ('管制', 'Nv'), ('3月', 'Nd'), ('7...",https://www.cna.com.tw/news/ahel/202203140235....,NaN
197,acn_20220314_5,2022-03-14,兩岸,香港確診增2.7萬例 本波死亡人數逾4千,香港今天新增COVID-19陽性病例2萬6908例，其中核酸確診1萬2040例，其餘1萬48...,1.0,"['其中默沙東藥廠的口服藥已開始處方給指定診所及安老院患者等', '首批輝瑞口服藥也會於今日...","[('香港', 6), ('疫情', 6), ('死亡率', 6), ('疫苗', 5), ...","['香港', '今天', '新增', 'COVID-19', '陽性', '病例', '2萬...","['香港', '陽性', '病例', '核酸', '確診', '快篩', '陽性', '疫情...","[NerToken(word='香港', ner='GPE', idx=(0, 2)), N...","[('香港', 'Nc'), ('今天', 'Nd'), ('新增', 'VJ'), ('C...",https://www.cna.com.tw/news/acn/202203140291.aspx,NaN


In [39]:
df_query.top_key_freq

137    [('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3),...
197    [('香港', 6), ('疫情', 6), ('死亡率', 6), ('疫苗', 5), ...
202    [('香港', 16), ('疫情', 10), ('長者', 9), ('死亡率', 7)...
206    [('香港', 5), ('疫情', 4), ('確診', 4), ('病例', 4), (...
209    [('香港', 7), ('人士', 6), ('施打', 6), ('疫苗', 6), (...
211    [('中國', 14), ('疫苗', 13), ('接種', 7), ('疫情', 6),...
Name: top_key_freq, dtype: object

### From the word_freq pairs, we sum frequency for each word

    How? 
    The best way is to use dict. Here is a simple example.

In [40]:
[('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3)]

[('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3)]

In [41]:
'''
{'陳時中': 7,
 '解封': 5,
 '疫情': 10,
 '檢疫期': 3}

 (key, value) key can't be duplicated.鍵值不能重複，轉成dict時不處理重複的，會被丟棄
'''

dict([('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3)])

{'陳時中': 7, '解封': 5, '疫情': 4, '檢疫期': 3}

In [42]:
#  (key, value) key can't be duplicated.鍵值不能重複，轉成dict時不處理重複的，會被丟棄
dict([('王金平', 2), ('吳敦義', 7), ('吳敦義', 5)])

{'王金平': 2, '吳敦義': 5}

In [43]:
c1 = Counter(dict([('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3)]))

In [44]:
c1

Counter({'陳時中': 7, '解封': 5, '疫情': 4, '檢疫期': 3})

In [45]:
c2 = Counter(dict([('馬英九', 2), ('陳時中', 7), ('蔡英文', 20)]))

In [46]:
c2

Counter({'蔡英文': 20, '陳時中': 7, '馬英九': 2})

In [47]:
c1+c2

Counter({'蔡英文': 20, '陳時中': 14, '解封': 5, '疫情': 4, '檢疫期': 3, '馬英九': 2})

In [48]:
### Now we can start to cout frequency from our word_freq pairs
df_query.iloc[0].top_key_freq

"[('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3), ('狀態', 3), ('指揮', 2), ('中心', 2), ('指揮官', 2), ('景點', 2), ('規模', 2), ('人流', 2), ('台灣', 2), ('縮短', 2), ('參選', 2), ('邊境', 1), ('管制', 1), ('重啟', 1), ('國門', 1), ('開放', 1), ('商務客', 1), ('國際', 1), ('冠狀病毒', 1), ('疾病', 1), ('香港', 1), ('人數', 1), ('中國', 1), ('深圳市', 1), ('東莞市', 1), ('封城', 1), ('學者', 1), ('專家', 1), ('政府', 1), ('訂定', 1), ('階段', 1), ('措施', 1), ('基準', 1), ('準備', 1), ('中央', 1), ('記者', 1), ('媒體', 1), ('預訂', 1), ('時程', 1), ('檢視', 1), ('能力', 1), ('疫苗', 1), ('涵蓋率', 1), ('國內外', 1), ('狀況', 1), ('規範', 1), ('整體', 1), ('社區', 1), ('風險', 1), ('期間', 1), ('台北', 1), ('市長', 1), ('柯文哲', 1), ('確診', 1), ('自由時報', 1), ('縣市長', 1), ('選舉', 1), ('民進黨', 1), ('政務官', 1), ('出戰', 1), ('台北市', 1), ('桃園市', 1), ('布局', 1), ('彈性', 1), ('發展', 1), ('回應', 1)]"

In [49]:
# Convert to dictionary
dict(eval(df_query.iloc[0].top_key_freq))

{'陳時中': 7,
 '解封': 5,
 '疫情': 4,
 '檢疫期': 3,
 '狀態': 3,
 '指揮': 2,
 '中心': 2,
 '指揮官': 2,
 '景點': 2,
 '規模': 2,
 '人流': 2,
 '台灣': 2,
 '縮短': 2,
 '參選': 2,
 '邊境': 1,
 '管制': 1,
 '重啟': 1,
 '國門': 1,
 '開放': 1,
 '商務客': 1,
 '國際': 1,
 '冠狀病毒': 1,
 '疾病': 1,
 '香港': 1,
 '人數': 1,
 '中國': 1,
 '深圳市': 1,
 '東莞市': 1,
 '封城': 1,
 '學者': 1,
 '專家': 1,
 '政府': 1,
 '訂定': 1,
 '階段': 1,
 '措施': 1,
 '基準': 1,
 '準備': 1,
 '中央': 1,
 '記者': 1,
 '媒體': 1,
 '預訂': 1,
 '時程': 1,
 '檢視': 1,
 '能力': 1,
 '疫苗': 1,
 '涵蓋率': 1,
 '國內外': 1,
 '狀況': 1,
 '規範': 1,
 '整體': 1,
 '社區': 1,
 '風險': 1,
 '期間': 1,
 '台北': 1,
 '市長': 1,
 '柯文哲': 1,
 '確診': 1,
 '自由時報': 1,
 '縣市長': 1,
 '選舉': 1,
 '民進黨': 1,
 '政務官': 1,
 '出戰': 1,
 '台北市': 1,
 '桃園市': 1,
 '布局': 1,
 '彈性': 1,
 '發展': 1,
 '回應': 1}

In [50]:
c1 = Counter(dict(eval(df_query.iloc[0].top_key_freq)))
c1.most_common(10)

[('陳時中', 7),
 ('解封', 5),
 ('疫情', 4),
 ('檢疫期', 3),
 ('狀態', 3),
 ('指揮', 2),
 ('中心', 2),
 ('指揮官', 2),
 ('景點', 2),
 ('規模', 2)]

In [51]:
c2 = Counter(dict(eval(df_query.iloc[1].top_key_freq)))
c2.most_common(10)

[('香港', 6),
 ('疫情', 6),
 ('死亡率', 6),
 ('疫苗', 5),
 ('接種', 4),
 ('安老院', 3),
 ('默沙東', 3),
 ('口服藥', 3),
 ('陽性', 2),
 ('病例', 2)]

In [52]:
counter = c1+c2
counter.most_common(10)

[('疫情', 10),
 ('陳時中', 7),
 ('香港', 7),
 ('疫苗', 6),
 ('死亡率', 6),
 ('解封', 5),
 ('接種', 4),
 ('檢疫期', 3),
 ('狀態', 3),
 ('中心', 3)]

In [53]:
# Advance operation for reference
# Using the itertools.groupby approach, you are summing the sorted groups based on first tuple elements.

# from itertools import groupby
# from operator import itemgetter

# my_list = [('a',2),('a',3),('b',3),('c',2),('b',4)]
# first = itemgetter(0)
# sums = [(k, sum(item[1] for item in tups_to_sum))
#         for k, tups_to_sum in groupby(sorted(my_list, key=first), key=first)]
# Outputs:

# [('a', 5), ('b', 7), ('c', 2)]

## Prepare wordcloud data

In [54]:
# Get related keywords by counting the top keywords of each news.
# Notice:  do not name function as  "get_related_keys",
# because this name is used in Django
def get_related_word_clouddata(df_query):

    # (1) Get wf_pairs by calling get_related_words().
    wf_pairs = get_related_words(df_query)
    
    # (2) cloud chart data
    # the minimum and maximum frequency of top words
    min_ = wf_pairs[-1][1]  # the last line is smaller
    max_ = wf_pairs[0][1]
    # text size based on the value of word frequency for drawing cloud chart
    textSizeMin = 20 # 最小字
    textSizeMax = 120 # 最大字
    # Scaling frequency value into an interval of from 20 to 120.
    clouddata = [{'text': w, 'size': int(textSizeMin + (f - min_) / (max_ - min_) * (textSizeMax - textSizeMin))}
                 for w, f in wf_pairs]

    return   wf_pairs, clouddata 

In [55]:
user_keywords = ['疫情','疫苗']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)

get_related_word_clouddata(df_query)

([('香港', 39),
  ('疫情', 34),
  ('疫苗', 32),
  ('接種', 20),
  ('中國', 16),
  ('長者', 15),
  ('死亡率', 13),
  ('病例', 12),
  ('確診', 8),
  ('病毒', 8),
  ('醫療', 8),
  ('陳時中', 7),
  ('安老院', 7),
  ('人士', 7),
  ('大學', 7),
  ('資源', 7),
  ('薛達', 6),
  ('施打', 6),
  ('解封', 5),
  ('開放', 5)],
 [{'text': '香港', 'size': 120},
  {'text': '疫情', 'size': 105},
  {'text': '疫苗', 'size': 99},
  {'text': '接種', 'size': 64},
  {'text': '中國', 'size': 52},
  {'text': '長者', 'size': 49},
  {'text': '死亡率', 'size': 43},
  {'text': '病例', 'size': 40},
  {'text': '確診', 'size': 28},
  {'text': '病毒', 'size': 28},
  {'text': '醫療', 'size': 28},
  {'text': '陳時中', 'size': 25},
  {'text': '安老院', 'size': 25},
  {'text': '人士', 'size': 25},
  {'text': '大學', 'size': 25},
  {'text': '資源', 'size': 25},
  {'text': '薛達', 'size': 22},
  {'text': '施打', 'size': 22},
  {'text': '解封', 'size': 20},
  {'text': '開放', 'size': 20}])

# Find paragraphs containing the keywords. 

    There may be too many related paragraphs, so we display only some of them on our Django website
    一一比對文章段落，找出關鍵詞所在的段落

## All-in-one function: Find related paragraphs

In [56]:
import re

# Step1: split paragraphs in text 先將文章切成一個段落一個段落
def cut_paragraph(text):
    paragraphs = text.split('。')  # 遇到句號就切開
    #paragraphs = re.split('。', text) # 遇到句號就切開
    #paragraphs = re.split('[。！!？?]', text) # 遇到句號(也納入問號、驚嘆號、分號等)就切開
    paragraphs = list(filter(None, paragraphs))
    return paragraphs


import re
# Find out all paragraphs where multiple keywords occur.
def get_same_para(df_query, user_keywords, cond, k=30):
    same_para=[]
    for text in df_query.content:
        #print(text)
        paragraphs = cut_paragraph(text)
        for para in paragraphs:
            para += "。"  # 在每段落文字後面加一個句號。
            # 判斷每個段落文字是否包含該關鍵字，and or分開判斷
            if cond == 'and':
                if all([kw in para for kw in user_keywords]):
                    same_para.append(para)  # 符合條件的段落para保存起來
            elif cond == 'or':
                if any([kw in para for kw in user_keywords]):
                    same_para.append(para)  # 符合條件的段落para保存起來
    return same_para[0:k]



# Step2: Select all paragraphs where multiple keywords occur.
def get_same_para(df_query, user_keywords, cond, k=30):
    same_para=[]
    for text in df_query.content:
        #print(text)
        paragraphs = cut_paragraph(text)
        for para in paragraphs:
            para += "。"
            if cond=='and':
                if all([re.search(kw, para) for kw in user_keywords]):
                    same_para.append(para)
            elif cond=='or':
                if any([re.search(kw, para) for kw in user_keywords]):
                    same_para.append(para)
    return same_para[0:k]


In [57]:
user_keywords = ['疫情','疫苗','香港']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)

In [58]:
len(df_query)

6

In [59]:
get_same_para(df_query, user_keywords, 'and', k=10)

['但可惜香港長者疫情接種率低，至今只有54.6%的80歲以上長者已接種1劑或以上疫苗。',
 '香港有超過90%染疫死者未完成接種2劑疫苗，而絕大部份人之前沒有感染過COVID-19病毒，所以在第5波疫情中沒有交叉保護作用。',
 '香港的COVID-19（2019冠狀病毒疾病）疫情嚴峻，官員表示，考慮擴大家居接種服務，上門為獨居或行動不便的長者和殘障人士施打疫苗。']

## A Step by step demonstration

## Step1: cut_paragraph() function

In [60]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,aipl_20220314_1,2022-03-14,政治,外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾,民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中...,0.01,"['外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外', '親赴外交部捐贈物資的民眾約173...","[('外交部', 14), ('民眾', 7), ('物資', 7), ('烏克蘭', 5)...","['民眾', '捐贈', '烏克蘭', '的', '愛心', '物資', '持續', '湧入...","['民眾', '烏克蘭', '愛心', '物資', '外交部', '收到', '外交部長',...","[NerToken(word='烏克蘭', ner='GPE', idx=(4, 7)), ...","[('民眾', 'Na'), ('捐贈', 'VD'), ('烏克蘭', 'Nc'), ('...",https://www.cna.com.tw/news/aipl/202203140364....,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...


In [61]:
text = df.content[0]
text

'民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝。外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日。外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問。根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜。外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物。送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾。募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部。外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力。外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意。'

In [62]:
# Step1: split paragraphs in text 先將文章切成一個段落一個段落
def cut_paragraph(text):
    paragraphs = text.split('。')  # 遇到句號就切開
    #paragraphs = re.split('。', text) # 遇到句號就切開
    #paragraphs = re.split('[。！!？?]', text) # 遇到句號(也納入問號、驚嘆號、分號等)就切開
    paragraphs = list(filter(None, paragraphs))
    return paragraphs


In [63]:
paragraphs = cut_paragraph(text)
paragraphs

['民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝',
 '外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日',
 '外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問',
 '根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜',
 '外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物',
 '送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾',
 '募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部',
 '外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力',
 '外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意']

In [64]:
text

'民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝。外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日。外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問。根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜。外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物。送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾。募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部。外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力。外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意。'

In [65]:
text.split('。')


['民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝',
 '外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日',
 '外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問',
 '根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜',
 '外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物',
 '送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾',
 '募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部',
 '外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力',
 '外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意',
 '']

In [66]:
paragraphs = text.split('。')
list(filter(None, paragraphs))


['民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝',
 '外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日',
 '外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問',
 '根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜',
 '外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物',
 '送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾',
 '募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部',
 '外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力',
 '外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意']

In [67]:
re.split('[。！!？?]', text) # regular expression 正規式 正則式

['民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝',
 '外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日',
 '外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問',
 '根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜',
 '外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物',
 '送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾',
 '募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部',
 '外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力',
 '外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意',
 '']

### How to cut paragraph? 

    (1) Approach 1:
    string.split() (it does not support regex)

    (2)Approach 2:
    Use regular expression正規式 re.split() 
    re.split() works fine

Simpe example to demonstrate the usage of re.split()

In [68]:
# Use string split().  It does not support regex.
# Split stentence using delimiter or separator '。'

text = '這是第1句話。這是第2句話?這是第3句話。'
text.split('。')

['這是第1句話', '這是第2句話?這是第3句話', '']

In [69]:
# It doesn't work. string split() method does not support regex
text.split('[。?]')

['這是第1句話。這是第2句話?這是第3句話。']

In [70]:
import re

In [71]:
re.split('。', "這是第1句話。這是，第2句話?這是--第3句話。")

['這是第1句話', '這是，第2句話?這是--第3句話', '']

In [72]:
# 如果納入多個符號去切割，必須用regular expression
# Here, [abc] will match if the string you are trying to match contains any of the a, b or c . 
# You can also specify a range of characters using - inside square brackets. [a-e] is the same as [abcde] . [1-4] is the same as [1234] .


In [73]:
# separator:。 ?
re.split('[。?]', "這是第1句話。這是，第2句話?這是--第3句話。")

['這是第1句話', '這是，第2句話', '這是--第3句話', '']

In [74]:
# "|" means or 可以加上｜去分隔開來，特別適用於當切割符號是由多個字組成時。
# separator:。 ?
re.split(r'[。|?]', "這是第1句話。這是，第2句話?這是--第3句話。")

['這是第1句話', '這是，第2句話', '這是--第3句話', '']

In [75]:
# "|" means or
# separator:。 ?
re.split('[句話|是]', "這是第1句話。這是，第2句話?這是--第3句話。")

['這', '第1', '', '。這', '，第2', '', '?這', '--第3', '', '。']

#### How to remove the empty elements?

In [76]:
# Do you notice the last element is an empty string?

In [77]:
result = re.split(r'[。?]', "這是第1句話。這是，第2句話?這是--第3句話。")

In [78]:
# Using Python filter function
filter(None, result)

In [79]:
list(filter(None, result))

['這是第1句話', '這是，第2句話', '這是--第3句話']

In [80]:
# An alternative way
[item for item in result if item]

['這是第1句話', '這是，第2句話', '這是--第3句話']

## Step 2: Find paragraphs containing keywords

### All-in-one function

In [81]:
import re
# Find out all paragraphs where multiple keywords occur.
def get_same_para(df_query, user_keywords, cond, k=30):
    same_para=[]
    for text in df_query.content:
        #print(text)
        paragraphs = cut_paragraph(text)
        for para in paragraphs:
            para += "。"  # 在每段落文字後面加一個句號。
            # 判斷每個段落文字是否包含該關鍵字，and or分開判斷
            if cond == 'and':
                if all([kw in para for kw in user_keywords]):
                    same_para.append(para)  # 符合條件的段落para保存起來
            elif cond == 'or':
                if any([kw in para for kw in user_keywords]):
                    same_para.append(para)  # 符合條件的段落para保存起來
    return same_para[0:k]


In [82]:
user_keywords = ['疫情','疫苗']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)

In [83]:
len(df_query)

6

In [84]:
get_same_para(df_query, user_keywords, 'and', k=10)

['陳時中說，無法預訂全面解封時程，這需要檢視自主應變能力、疫苗涵蓋率、國內外疫情狀況等；但，防疫規範整體是朝鬆綁邁進。',
 '但可惜香港長者疫情接種率低，至今只有54.6%的80歲以上長者已接種1劑或以上疫苗。',
 '香港有超過90%染疫死者未完成接種2劑疫苗，而絕大部份人之前沒有感染過COVID-19病毒，所以在第5波疫情中沒有交叉保護作用。',
 '他擔心港人已習慣每天逾萬例確診，隨著愈來愈多市民染疫後康復以及接種疫苗人口增加，加上抗疫疲勞，市民可能鬆懈，隨時讓令疫情反彈。',
 '香港的COVID-19（2019冠狀病毒疾病）疫情嚴峻，官員表示，考慮擴大家居接種服務，上門為獨居或行動不便的長者和殘障人士施打疫苗。',
 '他認為這些接下來應該加強做的事包括：老年人普遍的第3針接種，以及「更好的疫苗」與疫苗接種策略；可以廣泛供給的口服藥物；可以負擔得起的廣泛提供的居家檢測試劑；得到有效訓練和預演的分級診療策略；未來居家隔離的流程，下一次更大規模輸入與本土疫情疊加時所需的完整防控體系和充足醫療資源的準備等。']

### Step by step demonstration

In [85]:
df_query.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
137,ahel_20220314_18,2022-03-14,生活,陳時中：國門解封戒慎不恐懼 檢疫10天應該安全,邊境管制3月7日起鬆綁，指揮中心指揮官陳時中今天說，入境檢疫10天應該安全，現在談全面解封言...,0.0,"['台灣3月7日起將入境檢疫期縮短至10天', '中央流行疫情指揮中心指揮官陳時中今天在記者...","[('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3),...","['邊境', '管制', '3月', '7日', '起', '鬆綁', '，', '指揮',...","['邊境', '鬆綁', '指揮', '中心', '指揮官', '陳時中', '入境', '...","[NerToken(word='3月7日', ner='DATE', idx=(4, 8))...","[('邊境', 'Nc'), ('管制', 'Nv'), ('3月', 'Nd'), ('7...",https://www.cna.com.tw/news/ahel/202203140235....,NaN


In [86]:
text = df_query.content.iloc[0]
text

'邊境管制3月7日起鬆綁，指揮中心指揮官陳時中今天說，入境檢疫10天應該安全，現在談全面解封言之過早，清明節熱門景點不會有大規模管制，但適當人流管制仍屬必要。台灣3月7日起將入境檢疫期縮短至10天，重啟國門，先開放外籍商務客入境。國際間COVID-19（2019冠狀病毒疾病）疫情逐步趨緩；不過，香港染疫及死亡人數持續居高不下，中國繼深圳市後，東莞市今天起封城7天。最近台灣有學者專家建議，政府應及早訂定各階段防疫措施逐步解封的基準，為全面解封做準備，中央流行疫情指揮中心指揮官陳時中今天在記者回覆媒體時指出，陳時中說，目前談全面解封言之過早。陳時中說，無法預訂全面解封時程，這需要檢視自主應變能力、疫苗涵蓋率、國內外疫情狀況等；但，防疫規範整體是朝鬆綁邁進。目前社區仍有零星感染風險，他也說，清明連假期間，熱門景點雖然不會有大規模管制，但適當人流管制仍必要。對於台北市長柯文哲憂心3月7日起入境檢疫期縮短，17日開始會出現首波確診，陳時中表示，要戒慎不恐懼，一步一步來，檢疫期10天應該很安全。此外，據自由時報報導，今年年底縣市長選舉，不少民進黨政務官被點名出戰，陳時中被點名出征台北市、桃園市的布局彈性大，雖參選與否仍需視疫情發展而定，但已處於「備戰」狀態。陳時中對此回應，參選上並沒有在備戰狀態，但防疫一直都處於備戰狀態。'

In [87]:
paragraphs = cut_paragraph(text)
paragraphs

['邊境管制3月7日起鬆綁，指揮中心指揮官陳時中今天說，入境檢疫10天應該安全，現在談全面解封言之過早，清明節熱門景點不會有大規模管制，但適當人流管制仍屬必要',
 '台灣3月7日起將入境檢疫期縮短至10天，重啟國門，先開放外籍商務客入境',
 '國際間COVID-19（2019冠狀病毒疾病）疫情逐步趨緩；不過，香港染疫及死亡人數持續居高不下，中國繼深圳市後，東莞市今天起封城7天',
 '最近台灣有學者專家建議，政府應及早訂定各階段防疫措施逐步解封的基準，為全面解封做準備，中央流行疫情指揮中心指揮官陳時中今天在記者回覆媒體時指出，陳時中說，目前談全面解封言之過早',
 '陳時中說，無法預訂全面解封時程，這需要檢視自主應變能力、疫苗涵蓋率、國內外疫情狀況等；但，防疫規範整體是朝鬆綁邁進',
 '目前社區仍有零星感染風險，他也說，清明連假期間，熱門景點雖然不會有大規模管制，但適當人流管制仍必要',
 '對於台北市長柯文哲憂心3月7日起入境檢疫期縮短，17日開始會出現首波確診，陳時中表示，要戒慎不恐懼，一步一步來，檢疫期10天應該很安全',
 '此外，據自由時報報導，今年年底縣市長選舉，不少民進黨政務官被點名出戰，陳時中被點名出征台北市、桃園市的布局彈性大，雖參選與否仍需視疫情發展而定，但已處於「備戰」狀態',
 '陳時中對此回應，參選上並沒有在備戰狀態，但防疫一直都處於備戰狀態']

In [88]:
# Find out all paragraphs where multiple keywords occur.
user_keywords = ['疫情','疫苗']
cond='and'
same_para=[] # 存放含有關鍵字的段落
for para in paragraphs:
    para += "。" # 在每段落文字後面加一個句號。
    # 判斷每個段落文字是否包含該關鍵字，and or分開判斷
    if cond=='and': 
        if all([kw in para for kw in user_keywords]):
            same_para.append(para) # 符合條件的段落para保存起來
    elif cond=='or':
        if any([kw in para for kw in user_keywords]):
            same_para.append(para)  # 符合條件的段落para保存起來
same_para


['陳時中說，無法預訂全面解封時程，這需要檢視自主應變能力、疫苗涵蓋率、國內外疫情狀況等；但，防疫規範整體是朝鬆綁邁進。']

### multiple words in text (easier way)

In [89]:
para = '民進黨重用派系、酬庸人事的部分，他不再贅述；不過，他也批評國民黨，前總統馬英九的「交通幫」，沒有解決桃園機場跑道和漏水的問題，馬政府有許多人害怕跟宋楚瑜多接觸，擔心因此被老闆換掉，同樣是小氣、沒有肚量。'

In [90]:
user_keywords = ['馬英九','宋楚瑜']

In [91]:
any([kw in para for kw in user_keywords])


True

In [92]:
all([kw in para for kw in user_keywords])


True

In [93]:
user_keywords = ['馬英九','蔡英文']

In [94]:
any([kw in para for kw in user_keywords])


True

In [95]:
all([kw in para for kw in user_keywords])


False

### re.search(): An alternative way

In [96]:
# An alternative way for advanced users: using re.seach()
# Alternative approach using re.search() for reference
user_keywords = ['疫情','疫苗']
cond='and'
same_para=[] # 存放含有關鍵字的段落
for para in paragraphs:
    para += "。"
    if cond=='and':
        if all([re.search(kw, para) for kw in user_keywords]):
            same_para.append(para)
    elif cond=='or':
        if any([re.search(kw, para) for kw in user_keywords]):
            same_para.append(para)
same_para


['陳時中說，無法預訂全面解封時程，這需要檢視自主應變能力、疫苗涵蓋率、國內外疫情狀況等；但，防疫規範整體是朝鬆綁邁進。']

In [97]:
text = '民進黨重用派系、酬庸人事的部分，他不再贅述；不過，他也批評國民黨，前總統馬英九的「交通幫」，沒有解決桃園機場跑道和漏水的問題，馬政府有許多人害怕跟宋楚瑜多接觸，擔心因此被老闆換掉，同樣是小氣、沒有肚量。'

In [98]:
key = ['馬英九','蔡英文']

In [99]:
any([re.search(kw, text) for kw in user_keywords])

False

In [100]:
all([re.search(kw, text) for kw in user_keywords])

False

# views.py in Django website

To save memory, we just import df from the other app as follows.
from app_user_keyword.views import df

In [101]:
from django.shortcuts import render
from django.views.decorators.csrf import csrf_exempt
from django.http import JsonResponse

from datetime import datetime, timedelta
import pandas as pd
import math
import re
from collections import Counter

# (1) we can load data using read_csv() 自己app的csv檔案
# global variable
# df = pd.read_csv('dataset/cna_news_preprocessed.csv', sep='|')


# (2) we can load data using reload_df_data() function 隔壁app的csv檔案
# global variable
def load_df_data_v1():
    # global variable
    global  df
    df = pd.read_csv('app_user_keyword/dataset/cna_news_preprocessed.csv', sep='|')

# (3) df can be import from app_user_keyword 隔壁app的變數
# To save memory, we just import df from the other app as follows.
# from app_user_keyword.views import df

# (4) df can be import from app_user_keyword  隔壁app的變數
import app_user_keyword.views as userkeyword_views
def load_df_data():
    # import and use df from app_user_keyword 
    global df # global variable
    df = userkeyword_views.df

load_df_data()


# For the key association analysis
def home(request):
    return render(request, 'app_user_keyword_association/home.html')

# df_query should be global
@csrf_exempt
def api_get_userkey_associate(request):

    userkey = request.POST.get('userkey')
    cate = request.POST['cate']  # This is an alternative way to get POST data.
    cond = request.POST.get('cond')
    weeks = int(request.POST.get('weeks'))
    key = userkey.split()

    #global  df_query # global variable It's not necessary.

    df_query = filter_dataFrame_fullText(key, cond, cate, weeks)
    #print(key)
    print(len(df_query))

    if len(df_query) != 0:  # df_query is not empty
        newslinks = get_title_link_topk(df_query, k=25)
        related_words, clouddata = get_related_word_clouddata(df_query)
        same_paragraph = get_same_para(
            df_query, key, cond, k=30)  # multiple keywords
        num_articles=len(df_query) # total number of articles (stories, items)

    else:
        newslinks = []
        related_words = []
        same_paragraph = []
        clouddata = []
        num_articles=0

    response = {
        'num_articles': num_articles,
        'newslinks': newslinks,
        'related_words': related_words,
        'same_paragraph': same_paragraph,
        'clouddata': clouddata,
    }
    return JsonResponse(response)


# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_dataFrame_fullText(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()

    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() -
                  timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date)

    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: news category
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all(
            (qk in text) for qk in user_keywords))  # 寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any(
            (qk in text) for qk in user_keywords))  # 寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


# get titles and links from k pieces of news 
def get_title_link_topk(df_query, k=25):
    items = []
    for i in range( len(df_query[0:k]) ): # show only 10 news
        category = df_query.iloc[i]['category']
        title = df_query.iloc[i]['title']
        link = df_query.iloc[i]['link']
        photo_link = df_query.iloc[i]['photo_link']
        # if photo_link value is NaN, replace it with empty string 
        if pd.isna(photo_link):
            photo_link=''
        
        item_info = {
            'category': category, 
            'title': title, 
            'link': link, 
            'photo_link': photo_link
        }

        items.append(item_info)
    return items 

# Get related keywords by counting the top keywords of each news.
# Notice:  do not name function as  "get_related_keys",
# because this name is used in Django
def get_related_word_clouddata(df_query):

    # wf_pairs = get_related_words(df_query)
    # prepare wf pairs 
    counter=Counter()
    for idx in range(len(df_query)):
        pair_dict = dict(eval(df_query.iloc[idx].top_key_freq))
        counter += Counter(pair_dict)
    wf_pairs = counter.most_common(20) #return list format

    # cloud chart data
    # the minimum and maximum frequency of top words
    min_ = wf_pairs[-1][1]  # the last line is smaller
    max_ = wf_pairs[0][1]
    # text size based on the value of word frequency for drawing cloud chart
    textSizeMin = 20
    textSizeMax = 120
    # Scaling frequency value into an interval of from 20 to 120.
    clouddata = [{'text': w, 'size': int(textSizeMin + (f - min_) / (max_ - min_) * (textSizeMax - textSizeMin))}
                 for w, f in wf_pairs]

    return   wf_pairs, clouddata 


# Step1: split paragraphs in text 先將文章切成一個段落一個段落
def cut_paragraph(text):
    paragraphs = text.split('。')  # 遇到句號就切開
    #paragraphs = re.split('。', text) # 遇到句號就切開
    #paragraphs = re.split('[。！!？?]', text) # 遇到句號(也納入問號、驚嘆號、分號等)就切開
    paragraphs = list(filter(None, paragraphs))
    return paragraphs

# Step2: Select all paragraphs where multiple keywords occur.


def get_same_para(df_query, user_keywords, cond, k=30):
    same_para = []
    for text in df_query.content:
        #print(text)
        paragraphs = cut_paragraph(text)
        for para in paragraphs:
            para += "。"
            if cond == 'and':
                if all([re.search(kw, para) for kw in user_keywords]):
                    same_para.append(para)
            elif cond == 'or':
                if any([re.search(kw, para) for kw in user_keywords]):
                    same_para.append(para)
    return same_para[0:k]


    
print("app_user_keyword_association was loaded!")


ModuleNotFoundError: No module named 'app_user_keyword'

# For reference